In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys
sys.path.insert(1,'/home/shubham/Project/pos_tag/code/inference/distilbert_pos_tagger')

import argparse
import logging
import os
import random
from io import open
import re

import numpy as np
import pandas as pd

import pickle
import os

from nltk.tokenize import sent_tokenize
import json
import nltk, string,itertools
from nltk.corpus import stopwords

import torch
import torch.nn as nn
from bert_utils.modeling_bert import BertConfig, BertPreTrainedModel, BertModel
from bert_utils.tokenization_bert import BertTokenizer
import action_detector_prod as ad
#import action_detector as ad

import warnings
warnings.filterwarnings('ignore')

stop_words = set(stopwords.words('english'))
stop_words.add('hear')
stop_words.add('see')
stop_words = stop_words-set(['get','give','go','do','make'])

#get give go do make

stop_words_spacy = list("""
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere n't

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two


under until up unless upon us used using

various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves yeah okay
""".split()
)

stop_words = stop_words-set(['get','give','go','do','make','please'])
stop_words = set(list(stop_words)+list(stop_words_spacy))
#import spacy


#nlp = spacy.load("en_core_web_sm")

I0317 15:33:55.017485 140288362186560 file_utils.py:35] PyTorch version 1.2.0 available.


In [2]:
def st_get_candidate_phrases(text, pos_search_pattern_list=[
                                        r"""verbnoun:{<VB>+<.+>{0,2}<NN.*>+(<.+>{0,2}<JJ.*>*<NN.*>+)*}"""]):
        punct = set(string.punctuation)
        all_chunks = []

        for pattern in pos_search_pattern_list:
            all_chunks+=st_getregexChunks(text,pattern)

        candidates_tokens = [' '.join(word for word, pos, 
                    chunk in group).lower() 
                    for key, group in itertools.groupby(all_chunks, 
                    lambda_unpack(lambda word, pos, chunk: chunk != 'O')) if key]

        candidate_phrases = [cand for cand in candidates_tokens if cand not in stop_words and not all(char in punct for char in cand)]

        return candidate_phrases
    
def st_getregexChunks(text,grammar):

    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    print(grammar)
    print(all_chunks)
    print()

    return all_chunks

def lambda_unpack(f):
    return lambda args: f(*args)

def getValjsonDF(df):
    df.columns = list(pd.Series(df.columns).apply(lambda x: x.strip()))
    #df = df.drop(df.index[0])
    df = df['value']
    json_list = list()
    for i in df:
        if type(i)==str:
            json_list.append(json.loads(i))
    df = pd.DataFrame(json_list)
    return df

In [3]:
save_path = '/home/shubham/Project/action_item/models/prod/'
model_list = os.listdir(save_path)
config = BertConfig()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model_list = ['bert_ai_classifier_epc_1_step_1000_lr_3e-05_ws_1800.pkl']
file_name = save_path+model_list[0]
#file_name = '/home/shubham/hdd/shubham/projects/action_item_detection/Shubham/training/models/lr_3e-6_ws_1200/bert_ai_classifier_epc_5_step_3140_lr_3e-05_ws_1800.pkl'
#file_name = '/home/shubham/hdd/shubham/projects/action_item_detection/Shubham/training/models/lr_3e-6_ws_1200/seed/bert_ai_classifier_epc_45_step_2220_lr_3e-05_ws_1800_f1_0.81.pkl'
#file_name = '/home/shubham/hdd/shubham/projects/action_item_detection/Shubham/training/models/lr_3e-6_ws_1200/bert_ai_classifier_epc_6_step_3400_lr_3e-05_ws_1800_f10.787878787878788.pkl'
#file_name = '/home/shubham/hdd/shubham/projects/action_item_detection/Shubham/training/models/lr_3e-6_ws_1200/seed/precision/bert_ai_classifier_epc_33_step_1660_lr_3e-05_ws_1800_f1_0.9180327868852459.pkl'
#file_name = '/home/shubham/Project/action_item/models/bert_ai_classifier_epc_6_step_3400_lr_3e-05_ws_1800_f10.9411764705882354.pkl'
state_dict = torch.load(file_name,map_location='cpu')

model = ad.BertForActionItemDetection(config)
model.load_state_dict(state_dict)
model.eval()
print()


I0317 15:33:59.357710 140288362186560 tokenization_utils.py:249] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/shubham/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [4]:


#file_path = '../testset/7c41c54692534c0496f0901183b0aa94.csv'
#file_path = '/home/venkat/hdd/Venkat/action_item_detection/testset/1d6f6c2f-8dcf-4041-97da-cd0f861e7445.csv'
#file_path = '/home/venkat/hdd/Venkat/action_item_detection/testset/7c5a224f4fb54cdb83724612dd1088e8.csv'

import glob
file_list = glob.glob('/home/shubham/Project/data/call_transcripts/testset/*.csv')
#print(file_list)
sent_list =[]
sub_list = []
prob_list = []
all_ai = []
for index,file_path in enumerate(file_list[:]):
    #break
    df_tra= pd.read_csv(file_path,sep=',', error_bad_lines=False)
    try:
        df_tra = getValjsonDF(df_tra)
    except:
        continue
    print('##################')
    print()
    print('\x1b[1;31m'+'Meeting No: '+'\x1b[0m',index)
    print()
    df_tra = df_tra.sort_values(by='createdAt')
    text_list = list(df_tra['originalText'])
    ai_subj_list = []
    #text_list = pickle.load(open("../testset/49d9b53b-aebd-41ef-a8c8-5f545529a44a.pkl","rb"))
    
    for txt in text_list:
        #print(txt)
        input_ = {
            "segments": [
              { 
                "originalText": txt,
                "spokenBy": "Others",
                "id": "seg_id2",
                "confidence": 0.9178807
              }
            ]
        }
        #print('text:',txt)
        #print('')
        ai_detector = ad.ActionItemDetector(input_['segments'], model)
        action_items,decisions,questions = ai_detector.get_action_decision_subjects_list()
        _,ai_sent,_ = ai_detector.get_ai_candidates(txt)
        #print(txt)
        #sent_list = sent_list + junk
        #print(action_items)
        #for i in sent_list:
            #sent_list.append(i)
            #print(i,ai_detector.get_ai_probability(i))
            #print()
            #ai_subj_list.append(ai_obj['subject'])
            #print('Action item:',ai_obj['subject'])
            #print('#######')

#sent_dict = {'sentence':sent_list}
#sent_df = pd.DataFrame.from_dict(sent_dict)
#sent_df.to_csv('ai_meet_data.csv')
        """for ai_obj in questions:
            #print(txt)
            print()
            ai_subj_list.append(ai_obj['subject'])
            #sent_list.append(ai_obj['sent'])
            #sub_list.append(ai_obj['subject'])
            #prob_list.append(ai_obj['prob'])
            #print('Sent:',ai_obj['sent'],ai_obj['prob'] )
            for i in nltk.pos_tag(nltk.word_tokenize(ai_obj['subject'])):
                #print(i)
                pass
            print()
            print('Question:',ai_obj['subject'])
            print('#######')"""
            
        for ai_obj in action_items:
            #print(txt)
            print()
            ai_subj_list.append(ai_obj['subject'])
            #sent_list.append(ai_obj['sent'])
            sub_list.append(ai_obj['subject'])
            #prob_list.append(ai_obj['prob'])
            #print('Sent:',ai_obj['sent'],ai_obj['prob'] )
            for i in nltk.pos_tag(nltk.word_tokenize(ai_obj['subject'])):
                #print(i)
                pass
            print()
            all_ai.append(ai_obj['subject'])
            print('Action item:',ai_obj['subject'])
            print('#######')
        

##################

Meeting No:  0

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
##################

Meeting No:  1

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Visit the library and the presidential center
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bi

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
##################

Meeting No:  5

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
us

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: I mean if a transactional stuff comes through can you really continue to see organic constant?
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilst

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Yeah, can we pinpoint free to some ideal comparison between two sentences instead of the whole sentence level features?
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Use the refresh similarity as a start where the key phrase
#######
using bilstm
using bi

...

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Have new former clients and the laptops and then when we participate in the meeting so we will just use the client
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Skip ahead by clicking on the date appear
#######
using bilstm
using bilstm
using bilstm
usin

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm

using bilstm
using bilstm
using bilstm
using bilstm


Action item: Start and bunch of other things the bunch of things that we have to do forecast
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Support partial on the usage related changes
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bils

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Scale up your web applications to any number of users
#######


Action item: Get discounts from 20 percent
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm

using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action item: Find a react developer someone who can work with 

In [5]:
df = pd.read_csv('/home/shubham/Downloads/dialpad_transcript_20200311_1242.csv')
txt_ls = df['content'].values.tolist()
for txt in txt_ls:
    #break
    #txt = "You have to rely on key phrases and segments."
    #txt = "So this will eventually be used to Replace the current mind like the way it's very superior to the current mind, but we need to think of how to once we change the way my mind works."
    input_ = {
            "segments": [
              { 
                "originalText": txt,
                "spokenBy": "Others",
                "id": "seg_id2",
                "confidence": 0.9178807
              }
            ]
        }
    ai_subj_list = []
    ai_detector = ad.ActionItemDetector(input_['segments'], model)
    action_items,decisions,questions = ai_detector.get_action_decision_subjects_list()
    _,ai_sent,_ = ai_detector.get_ai_candidates(txt)
    score = ai_detector.get_ai_probability(txt)
    #sent_list = sent_list + junk
    #print(junk,score)
    for ai_obj in action_items:
                #print(txt)
                print()
                ai_subj_list.append(ai_obj['subject'])
                #sent_list.append(ai_obj['sent'])
                #sub_list.append(ai_obj['subject'])
                #prob_list.append(ai_obj['prob'])
                #print('Sent:',ai_obj['sent'],ai_obj['prob'] )
                """for i in nltk.pos_tag(nltk.word_tokenize(txt)):
                    print(i)
                for i in nltk.pos_tag(nltk.word_tokenize(ai_obj['subject'])):
                    print(i)"""
                print()
                print('Action:',ai_obj['subject'])
                print('#######')


using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


Action: Achieve better authorization across replication system
#######
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm
using bilstm


In [6]:
    txt = "i am thinking that let's take an action"

    """for i in nltk.pos_tag(nltk.word_tokenize(txt)):
                    print(i)"""
    input_ = {
            "segments": [
              { 
                "originalText": txt,
                "spokenBy": "Others",
                "id": "seg_id2",
                "confidence": 0.9178807
              }
            ]
        }
    ai_subj_list = []
    ai_detector = ad.ActionItemDetector(input_['segments'], model)
    action_items,decisions,questions = ai_detector.get_action_decision_subjects_list()
    _,ai_sent,_ = ai_detector.get_ai_candidates(txt)
    score = ai_detector.get_ai_probability(txt)
    #sent_list = sent_list + junk
    #print(junk,score)
    for ai_obj in action_items:
                #print(txt)
                print()
                ai_subj_list.append(ai_obj['subject'])
                #sent_list.append(ai_obj['sent'])
                #sub_list.append(ai_obj['subject'])
                #prob_list.append(ai_obj['prob'])
                #print('Sent:',ai_obj['sent'],ai_obj['prob'] )
                for i in nltk.pos_tag(nltk.word_tokenize(txt)):
                    print(i)
                """for i in nltk.pos_tag(nltk.word_tokenize(ai_obj['subject'])):
                    print(i)"""
                print()
                print('Action:',ai_obj['subject'])
                print('#######')


('i', 'NN')
('am', 'VBP')
('thinking', 'VBG')
('that', 'IN')
('let', 'NN')
("'s", 'POS')
('take', 'VB')
('an', 'DT')
('action', 'NN')

Action: i am thinking that let's take an action
#######


In [7]:
sent = "Let's work on this"
print(nltk.word_tokenize(sent))
for i in nltk.pos_tag(nltk.word_tokenize(txt)):
    print(i)

['Let', "'s", 'work', 'on', 'this']
('i', 'NN')
('am', 'VBP')
('thinking', 'VBG')
('that', 'IN')
('let', 'NN')
("'s", 'POS')
('take', 'VB')
('an', 'DT')
('action', 'NN')


In [8]:
l = list(range(10))
def check():
    for i in l:
        if i not in l:
            print(l,i)
check()

In [9]:
#print(type(nltk.help.upenn_tagset()))


In [10]:
"""print(len(all_ai))
with open('bert_ai_prod.pkl','wb') as f:
    pickle.dump(all_ai,f)"""

"print(len(all_ai))\nwith open('bert_ai_prod.pkl','wb') as f:\n    pickle.dump(all_ai,f)"

In [11]:
"""with open('bert_ai_prod.pkl','rb') as g:
    bert_dump  = pickle.load(g)
with open('nltk_ai_prod.pkl','rb') as f:
    nltk_dump  = pickle.load(f)"""

"with open('bert_ai_prod.pkl','rb') as g:\n    bert_dump  = pickle.load(g)\nwith open('nltk_ai_prod.pkl','rb') as f:\n    nltk_dump  = pickle.load(f)"

In [12]:
"""for i in (set(nltk_dump)-set(bert_dump)):
    print(i)
    print()"""

'for i in (set(nltk_dump)-set(bert_dump)):\n    print(i)\n    print()'

In [13]:
"""for i in (set(bert_dump)-set(nltk_dump)):
    print(i)
    print()"""

'for i in (set(bert_dump)-set(nltk_dump)):\n    print(i)\n    print()'

In [14]:
from bert_pos_tagger import BertPosTagger as bpt
btag = bpt('/home/shubham/Project/pos_tag/models/epoch-2/pytorch_model.bin')
from distilbert_pos_tagger import DistilBertPosTagger as dbpt
dbtag = dbpt('/home/shubham/Project/pos_tag/models/distil/pytorch_model.bin')


I0317 16:06:41.273815 140288362186560 modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0317 16:06:41.286562 140288362186560 modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0317 16:06:43.176158 140288362186560 file_utils.py:35] PyTorch version 1.2.0 available.


In [15]:
text = "' Hello my name is Shubham.'"

bert = btag.get_sent_pos_tags(text)
Distbert = dbtag.get_sent_pos_tags(text)

for i in nltk.pos_tag(nltk.word_tokenize(text)):
    print(i)

print('------------')

for i in bert:
    print(i)
    
print('------------')

for i in Distbert:
    print(i)

bert [101]
("'", 'POS')
('Hello', 'NNP')
('my', 'PRP$')
('name', 'NN')
('is', 'VBZ')
('Shubham', 'NNP')
('.', '.')
("'", "''")
------------
("'", 'PUNC')
('Hello', 'NNP')
('my', 'PRP$')
('name', 'NN')
('is', 'VBZ')
('Shubham', 'NNP')
('.', 'PUNC')
("'", 'PUNC')
------------
("'", 'PUNC')
('Hello', 'NNP')
('my', 'PRP$')
('name', 'NN')
('is', 'VBZ')
('Shubham', 'NNP')
('.', 'PUNC')
("'", 'PUNC')


In [16]:
import transformers

In [17]:
print(transformers.__file__)

/home/shubham/anaconda3/lib/python3.6/site-packages/transformers/__init__.py


In [18]:
ls = ["""   部分の装甲を強化するかを決定するために、被弾しながらも帰還した機体を調査しました。しかし撃墜された機体の被弾箇所については考慮しなかったため、最も攻撃を受けやすい機体部位を見落としました。 「生存者バイアス」とも呼ばれるこの現象は、成功を収めた結果のみを詳しく調査し、それに付随する失敗を見過ごしがちな人間の性質を表しています。私たちはあらゆるデータを検討しないままに、意見を採用したり、ビジネスを構築したり、意思決定を行ったりして、失敗に陥りやすいのです。 生存バイアスの詳細については、以下のTED Talkをご覧ください。 それでは、ビジネスにおける生存バイアスを回避するにはどうすればよいでしょうか？ 以下に、生存バイアスとしてよくある9つの例と、それぞれの生存バイアスの防止策をご紹介します。 この例はビジネスに直接関係するものではありませんが、生存バイアスという考え方の起源と見なされることが多いため、最初にご紹介しましょう。 第二次世界大戦中、連合軍は戦闘機の装甲を強化する必要があると考えました。しかし、材料に限りがあるため、機体全体に追加装甲を施すことはできません。そこで、専門家は最も攻撃を受けやすく、装甲を強化することで最大の効果が得られる部位を特定しなければなりませんでした。 追加装甲の部位を決定するうえで、専門家は被弾しながらも無事に帰還した機体を調査しました。調査の結果、帰還した機体のエンジンやコックピットには弾痕がなかったため、必然的にエンジンとコックピット以外のすべての部位の装甲を強化しようという結論に至りました。 幸いにも、数学者のAbraham Wald氏がこの計画の欠陥を指摘しました。無事に帰還した機体しか分析していないことが問題だと主張したのです。つまり被弾した箇所が機体にとって致命的ではなかったからこそ、その戦闘機は撃墜に至らずに済んだわけです。Wald氏は、帰還した機体の弾痕がなかった部位に装甲を追加することを軍に提案しました。 撃墜された原因を分析することで、Wald氏は生存バイアスという考え方を生み出し、このことは多くの操縦士の命を救うことにつながったと考えられます。 Googleで「大学を中退して成功した創業者」と検索してみると、世界屈指の有名企業がいくつかヒットします。ジョブズ氏、ゲイツ氏、ザッカーバーグ氏はいずれも優れたアイデアを胸に躍進し、奇跡的に成功を収めた起業家です。 しかし「多大な努力＝必ず成功」という先入観を持ってしまうと、重大な事実を見落とすことになります。大学中退者全体のうち、不運に見舞われている中退者の人数は決して少なくないはずです。 尊敬を集める創業者となりえたのは確かに懸命な努力があってこそですが、特定の状況でしか起こらなかった数々の出来事によって成功への道が開かれたとも考えられます。実際に、ある調査によると、米国で並外れた成功を収めた実業家の大部分（具体的には94％）は大学を卒業しています。 成功を収めるうえで学位は必要ないという仮説は、生存バイアスの一例です。もちろん例外はありますが、入手可能なすべての事実を検討してから結論を下すことが重要です。 「成功する人々の毎朝の習慣」、「すべての億万長者に共通する6つの特徴」、「ジェフ・ベゾスが語る成功に必要なたった1つの要因」、こういった記事をついつい読んでしまったという経験はないでしょうか？ かく言う私も、恥ずかしながら、似たような記事に幾度となく飛びついてきました。 私たちは、崇拝する人物について学べば同じように成功できると信じてやまないのです。ただし問題は、こういった記事でも、詳細な伝記でさえも、成功の追体験に必要なすべてのデータが提示されているわけではないという点です。一般読者は、背景にある変動要因を見落としたまま、不十分なデータを基に意思決定を行ってしまいます。 ベゾス氏がAmazonを立ち上げる以前から、同じような事業に挑戦して失敗した人はおそらく何百人もいるでしょう。しかし、そういった人に関する書籍はなかなか見つかりません。成功を収めた人物の成功を再現しようとしても、同様の結果が得られるとは限らないということを肝に銘じておきましょう。成功は努力以外に、周囲の環境や要因に大きく左右される場合もあるのです。 「次なるNetflix」、「○○業界のUber」。話題のスタートアップ企業に対してこんな表現が使用されているのを目にしたことがあるでしょう。しかし、米国中小企業庁によると、2年間存続した企業は全体のわずか3分の2で、5年以上だと全体の半分、10年以上になると全体の3分の1になります。 勝算は低く、成功を収めた企業のビジネスモデルに倣うだけでは、同様の成功を収められるとは限りません。 この手の生存バイアスの影響を受けて、多くの創業者は現在の市場、顧客、成長段階に適していないビジネスモデルを採用しようとします。 新規事業を立ち上げる場合には、尊敬する企業からインスピレーションを得ることも大切ですが、自社が置かれている現在の市場を評価して、アイデアを成功させるためにどのように進化するべきかを理解することが重要です。 デジタル ビデオ レコーダー（DVR）のTiVoが現在も事業を継続していることをご存じですか？ このDVRは、いち早く市場に出回り、製品としては現在でも優れています。全盛期にはTiVoが動詞として使われるほど大人気で、バラエティー番組を録画したりして5年以上はTiVoを使用していたという人も少なくありません。 TiVoの例は、優れた製品であっても、使いやすさやブランドロイヤルティーの勝負が始まり、市場に製品があふれる段階になると淘汰されることをよく表しています。高品質の製品やサービスを開発したからといって、当然のように顧客からの支持を集められると考えてはいけません（その典型が、わずか3年前にSketchersが市場シェアでAdidasを上回った一件です）。 生存バイアスを回避するためには、直接目に見えない他の変動要因も考慮する必要があります。 「成功する人々の毎朝の習慣」といった記事と同じくらいよく目にするのが、ある1つの変革によって劇的な成果を挙げた企業の事例と、その具体的な方法について説明する記事です。 しかし、生存バイアスから教訓を得るなら、同じような変革を試みてそれほど成果が得られなかった企業についても考慮する必要があります。 営業チームXがこのEメールテンプレートを使用して成約率を35％向上できたのが事実だとします。しかし、それに加えて新しい営業幹部を招き入れたり、報酬制度を改定したり、何か月も滞っていた数件の大口取引が偶然にも成約に至ったり、といった事実があっても、記事では言及されていない可能性があります。 このテンプレートの使用が成約率35％向上の唯一の要因だと考え、自社でも同じような効果を期待すると、高い確率で失望に終わることになります。 既存の顧客基盤のみに基づいて年間経常収益（ARR）を試算するなら、実際の収益が想定を下回るという厳しい現実を覚悟しなければいけません。 事業計画の中心となるのは確かに既存の顧客基盤ですが、生存バイアスを回避するためには、前年同期の顧客解約率を調査して、ARRの計算でも考慮に入れる必要があります。 そうすれば、想定される収益をより正確に把握できるため、将来的に見込みが外れて失望したり想定外のキャッシュフロー不足に陥ったりする事態を回避できます。 解約率を下げるために、満足度の低い顧客からクレームが来たら真摯に対応しようとしても、その顧客は既に失ったも同然という可能性が高いのです。ある調査では、SaaSユーザーの約半数はアカウントを無効化する2週間以上前に解約の手続きを開始しているという点が指摘されています。 したがって、顧客から苦情メールが送られてこないからといって、顧客が完全に満足しているものと思い込むのは危険です。Net Promoter Score®（NPS®）などのツールを使用して、顧客満足度と成長の可能性を測定しましょう。 満足度の高い顧客の声を把握することで、不安の兆候を早期に発見し、実際に問題が発生した場合でも、顧客が解約を決める前に先回りして対応することができます。 必要以上に機能を追加してしまう「フィーチャークリープ」という現象は大半の創業者が一度は直面する課題であり、顧客から要望があったという理由だけで、事業領域を広げて製品やサービスの機能を開発または実装し始めた場合に起こります。 市場、競合他社、顧客のニーズを把握することは確かに重要ですが、自社の本来の事業目的を見失わないことも等しく重要です。 機能を追加したところで、自社の主力製品／サービスの成長の妨げになるのでは、仮に少しは新規の顧客を獲得できたとしても、時間と労力に見合うほどの成果とは言えません。 機能を追加するメリットにだけ目を向けるのではなく、生存バイアスを考慮して、機能を追加することの影響をあらゆる角度から吟味してください。 生存バイアスを考慮することで、これまでとは違った新たな観点から問題を考えることができます。これは、多くの起業家が得意とする領域でしょう。意思決定を行う前に、入手可能なあらゆるデータを検討しましょう。それこそがビジネスの成否を決める鍵になるかもしれません。."""]
#ls = ['部']
for i in ls:
    print()
    #print(nltk.word_tokenize(i))
    #print(len(tokenizer.tokenize(i)))
    for j in dbtag.get_sent_pos_tags(i):
        #print(i)
        print(j)


('部分の装甲を強化するかを決定するために、被弾しながらも帰還した機体を調査しました。しかし撃墜された機体の被弾箇所については考慮しなかったため、最も攻撃を受けやすい機体部位を見落としました。', 'NNP')
('「生存者バイアス」とも呼ばれるこの現象は、成功を収めた結果のみを詳しく調査し、それに付随する失敗を見過ごしがちな人間の性質を表しています。私たちはあらゆるデータを検討しないままに、意見を採用したり、ビジネスを構築したり、意思決定を行ったりして、失敗に陥りやすいのです。', 'NNP')
('生存バイアスの詳細については、以下のTED', 'NNP')
('Talkをご覧ください。', 'NN')
('それでは、ビジネスにおける生存バイアスを回避するにはどうすればよいでしょうか？', 'NN')
('以下に、生存バイアスとしてよくある9つの例と、それぞれの生存バイアスの防止策をご紹介します。', 'CD')
('この例はビジネスに直接関係するものではありませんが、生存バイアスという考え方の起源と見なされることが多いため、最初にご紹介しましょう。', 'NNP')
('第二次世界大戦中、連合軍は戦闘機の装甲を強化する必要があると考えました。しかし、材料に限りがあるため、機体全体に追加装甲を施すことはできません。そこで、専門家は最も攻撃を受けやすく、装甲を強化することで最大の効果が得られる部位を特定しなければなりませんでした。', 'NNP')
('追加装甲の部位を決定するうえで、専門家は被弾しながらも無事に帰還した機体を調査しました。調査の結果、帰還した機体のエンジンやコックピットには弾痕がなかったため、必然的にエンジンとコックピット以外のすべての部位の装甲を強化しようという結論に至りました。', 'NNP')
('幸いにも、数学者のAbraham', 'NNP')
('Wald氏がこの計画の欠陥を指摘しました。無事に帰還した機体しか分析していないことが問題だと主張したのです。つまり被弾した箇所が機体にとって致命的ではなかったからこそ、その戦闘機は撃墜に至らずに済んだわけです。Wald氏は、帰還した機体の弾痕がなかった部位に装甲を追加することを軍に提案しました。', 'VBN')
('撃墜された原因を分析することで、Wald氏は生存バイアスという考え方を生

In [19]:
!python --version

Python 3.6.8 :: Anaconda, Inc.


In [20]:
import platform
platform.python_version()

'3.6.8'

In [21]:
print(tokenizer.encode("[CLS]"))

[101]
